In [14]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [15]:
screenshot_path = "traffic_sim/screenshots"
screenshot_list = os.listdir(screenshot_path)
screenshot_list = sorted(list(map(lambda x: os.path.join(screenshot_path, x), screenshot_list)))

In [16]:
labels_path = "traffic_sim/labels"

for filename in os.listdir(labels_path):
    file_path = os.path.join(labels_path, filename)
    os.remove(file_path)

In [17]:
# Load the image
image_path = '/Users/sudharsangopalakrishnan/SmarTransit/traffic_sim/images/intersection_placeholder.png'
image = cv2.imread(image_path)

# Invert the colors
inverted_image = 255 - image

# Save the modified image
cv2.imwrite('/Users/sudharsangopalakrishnan/SmarTransit/traffic_sim/images/intersection_placeholder_inverted.png', inverted_image)

True

In [18]:
directions = ["up", "down", "left", "right"]
vehicles = ["bike", "bus", "car", "truck"]
thresholds = [0.7 for _ in range(len(directions) * len(vehicles))]
threshold_dict = {}
IoU_threshold = 0.5  # Threshold for non-max suppression

# Populate the threshold dictionary
count = 0
for d in directions:
    for v in vehicles:
        threshold_dict[f"{d}-{v}"] = thresholds[count]
        count += 1

# Process each screenshot in the list
for index in range(1, len(screenshot_list), 2):
    print(f"Image {index + 1}:")
    path = screenshot_list[index]
    labels = []

    main_image = cv2.imread(path)
    for d in directions:
        for cls, v in enumerate(vehicles):
            template = cv2.imread(f"traffic_sim/images/{d}/{v}.png")

            # Convert images to grayscale
            main_gray = cv2.cvtColor(main_image, cv2.COLOR_BGR2GRAY)
            template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

            # Perform template matching
            result = cv2.matchTemplate(main_gray, template_gray, cv2.TM_CCOEFF_NORMED)

            # Get locations above the threshold
            threshold = threshold_dict[f"{d}-{v}"]
            locations = np.where(result >= threshold)
            print(f"{d}-{v}")
            print(np.shape(locations))

            # Template dimensions
            template_height, template_width = template_gray.shape[:2]
            main_height, main_width = main_gray.shape[:2]

            # Track matched bounding boxes
            matched_bboxes = []

            # Loop through all matches
            for pt in zip(*locations[::-1]):
                top_left = pt
                center_x = (top_left[0] + template_width / 2) / main_width
                center_y = (top_left[1] + template_height / 2) / main_height
                bbox_width = template_width / main_width
                bbox_height = template_height / main_height

                # Check for overlaps with previous matches
                is_duplicate = False
                for bbox in matched_bboxes:
                    x1, y1, x2, y2 = bbox
                    new_x1 = top_left[0]
                    new_y1 = top_left[1]
                    new_x2 = top_left[0] + template_width
                    new_y2 = top_left[1] + template_height

                    # Compute IoU
                    inter_x1 = max(x1, new_x1)
                    inter_y1 = max(y1, new_y1)
                    inter_x2 = min(x2, new_x2)
                    inter_y2 = min(y2, new_y2)

                    inter_width = max(0, inter_x2 - inter_x1)
                    inter_height = max(0, inter_y2 - inter_y1)
                    intersection = inter_width * inter_height
                    union = (template_width * template_height) * 2 - intersection
                    IoU = intersection / union if union > 0 else 0

                    # Mark as duplicate if IoU exceeds the threshold
                    if IoU > IoU_threshold:
                        is_duplicate = True
                        break

                if not is_duplicate:
                    # Add to labels and matched_bboxes
                    labels.append([cls, center_x, center_y, bbox_width, bbox_height])
                    matched_bboxes.append((top_left[0], top_left[1],
                                           top_left[0] + template_width,
                                           top_left[1] + template_height))

    # Save labels to a text file
    output_path = f"{path.split('/')[0]}/labels/{path.split('/')[-1][:-4]}.txt"
    with open(output_path, "w") as f:
        for label in labels:
            f.write(f"{label[0]} {label[1]} {label[2]} {label[3]} {label[4]}\n")

Image 2:
up-bike
(2, 1)
up-bus
(2, 0)
up-car
(2, 0)
up-truck
(2, 0)
down-bike
(2, 0)
down-bus
(2, 0)
down-car
(2, 0)
down-truck
(2, 0)
left-bike
(2, 0)
left-bus
(2, 0)
left-car
(2, 0)
left-truck
(2, 0)
right-bike
(2, 0)
right-bus
(2, 0)
right-car
(2, 0)
right-truck
(2, 0)
Image 4:
up-bike
(2, 1)
up-bus
(2, 0)
up-car
(2, 0)
up-truck
(2, 0)
down-bike
(2, 0)
down-bus
(2, 0)
down-car
(2, 0)
down-truck
(2, 0)
left-bike
(2, 0)
left-bus
(2, 0)
left-car
(2, 0)
left-truck
(2, 0)
right-bike
(2, 0)
right-bus
(2, 0)
right-car
(2, 0)
right-truck
(2, 0)
Image 6:
up-bike
(2, 2)
up-bus
(2, 0)
up-car
(2, 0)
up-truck
(2, 0)
down-bike
(2, 0)
down-bus
(2, 0)
down-car
(2, 0)
down-truck
(2, 0)
left-bike
(2, 0)
left-bus
(2, 0)
left-car
(2, 0)
left-truck
(2, 0)
right-bike
(2, 0)
right-bus
(2, 0)
right-car
(2, 0)
right-truck
(2, 0)
Image 8:
up-bike
(2, 2)
up-bus
(2, 0)
up-car
(2, 0)
up-truck
(2, 0)
down-bike
(2, 0)
down-bus
(2, 0)
down-car
(2, 0)
down-truck
(2, 0)
left-bike
(2, 1)
left-bus
(2, 0)
left-car
(2, 

In [ ]:
'''
up:
    bike: 0.7
    bus: 0.9
    car: 0.7
    truck: 0.7
down:
    bike: 0.7
    bus: 0.9
    car: 0.9
    truck: 0.9
left:
    bike: 0.7
    bus: 0.7
    car: 0.7
    truck: 0.7
right:
    bike: 0.7
    bus: 0.9
    car: 0.9
    truck: 0.9

'''